In [35]:
import pandas as pd
import numpy as np

## Institutes

In [36]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df = df.drop_duplicates()
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII181371,Sulmona,Casa di reclusione,323,8,434,225,255,23,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,63,555,316,336,27,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181870,Verbania,Casa circondariale,53,0,83,43,44,5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272,7,370,203,214,24,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


### Remove outliers

In [37]:
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII181371,Sulmona,Casa di reclusione,323,8,434,225,255,23,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,63,555,316,336,27,30,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181870,Verbania,Casa circondariale,53,0,83,43,44,5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII180348,Roma Rebibbia femminile,Casa circondariale - Germana Stefanini,272,7,370,203,214,24,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


In [38]:
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'])
df = df.sort_values(by='dati aggiornati al')

In [39]:
# 1. Group by 'dati aggiornati al' and sum 'posti regolamentari'
daily_total_posti = df.groupby('dati aggiornati al')['posti regolamentari'].sum()
daily_total_posti

dati aggiornati al
2024-10-05    51196
2024-10-06    51196
2024-10-07    51196
2024-10-09    51195
2024-10-10    51195
              ...  
2025-06-18    51284
2025-06-19    51284
2025-06-22    51276
2025-06-23    51276
2025-06-24    51276
Name: posti regolamentari, Length: 207, dtype: int64

In [40]:
# 2. Calculate the percentage difference from the previous day's total
# pct_change() is perfect for this.
daily_total_posti_diff_pct = daily_total_posti.pct_change().abs()
daily_total_posti_diff_pct.tail(20)

dati aggiornati al
2025-05-30    0.000000
2025-05-31    0.000019
2025-06-01    0.000000
2025-06-02    0.000000
2025-06-03    0.000000
2025-06-04    0.000000
2025-06-06    0.000000
2025-06-07    0.000000
2025-06-08    0.000000
2025-06-09    0.000000
2025-06-10    0.000000
2025-06-11    0.000000
2025-06-12    0.000000
2025-06-15    0.000000
2025-06-16    0.000000
2025-06-18    0.000000
2025-06-19    0.000000
2025-06-22    0.000156
2025-06-23    0.000000
2025-06-24    0.000000
Name: posti regolamentari, dtype: float64

In [41]:
# Define the percentage threshold for outlier
percentage_threshold = 0.30

In [42]:
bad_dates = daily_total_posti_diff_pct[daily_total_posti_diff_pct > percentage_threshold].index.tolist()
bad_dates

[]

In [43]:
# Removing false positives
# This assumes the pattern: [True Outlier, False Positive, True Outlier, False Positive, ...]
dates_to_remove = [date for i, date in enumerate(bad_dates) if i % 2 == 0]
dates_to_remove

[]

In [44]:
df_cleaned = df[~df['dati aggiornati al'].isin(dates_to_remove)].copy()
df_cleaned

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
121,MII172508,Bologna,Casa circondariale - Rocco D'Amato,503,14,823,425,462,29,35,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
122,MII179842,Piacenza,Casa circondariale - San Lazzaro,416,9,494,255,215,19,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
123,MII174686,Ferrara,Casa circondariale - Costantino Satta,244,0,382,181,194,17,19,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
124,MII180001,Rimini,Casa circondariale,118,0,155,109,127,14,17,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39164,MII179364,Parma,Casa di reclusione,655,33,779,342,423,1,0,...,443.0,145.0,16.0,515.0,515.0,515.0,2025-06-24,2025-05-31,2025-05-31,2025-06-24
39165,MII158945,Arbus,Casa di reclusione - località Is Arenas,176,50,90,71,81,12,20,...,40.0,0.0,0.0,40.0,40.0,40.0,2025-06-24,2025-05-31,2025-05-31,2025-06-24
39166,MII179237,Opera,Casa di reclusione - Iª casa,918,18,1367,512,628,39,54,...,230.0,625.0,4.0,818.0,226.0,236.0,2025-06-24,2025-05-31,2025-05-31,2025-06-24
39157,MII152284,Genova Marassi,Casa circondariale,535,0,658,319,336,28,36,...,128.0,175.0,3.0,213.0,100.0,0.0,2025-06-24,2025-05-31,2025-05-31,2025-06-24


In [45]:
df_cleaned.to_csv('../outputs/clean/institutes.csv', index=False, encoding='UTF-8-sig')
df = pd.read_csv('../outputs/clean/institutes.csv')


## Most recent data

In [46]:
# Get the most recent update
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id istituto')['dati aggiornati al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti disponibili'] = df_most_recent['posti regolamentari'] - df_most_recent['posti non disponibili']
df_most_recent['tasso di affollamento'] = round(((df_most_recent['totale detenuti'] / df_most_recent['posti disponibili'])*100),0)

In [47]:
df_most_recent.value_counts(['dati aggiornati al'])

dati aggiornati al
2025-06-24            190
dtype: int64

In [48]:
df_most_recent.columns

Index(['id istituto', 'nome istituto', 'tipo istituto', 'posti regolamentari',
       'posti non disponibili', 'totale detenuti',
       'polizia penitenziaria - effettivi', 'polizia penitenziaria - previsti',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'educatori - effettivi', 'educatori - previsti', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'dati aggiornati al',
       'personale polizia penitenziaria aggiornato al',
       'personale amministrativo aggiornato al',
       'data di aggiornamento spazi detentivi', 'posti disponibili',
       'tasso di affollamento'],
      dtype='object')

In [49]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id istituto'})

merged_df = pd.merge(df_most_recent, df_info, on='id istituto')

# Adding additional columns
merged_df['posti disponibili'] = merged_df['posti regolamentari'] - merged_df['posti non disponibili']

merged_df['tasso di affollamento'] = round((merged_df['totale detenuti'] / merged_df['posti disponibili']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id istituto',
    'nome istituto',
    'tasso di affollamento',
    'indirizzo',
    'tipo istituto',
   'posti regolamentari',
    'posti non disponibili',
    'posti disponibili',
    'totale detenuti',
    'dati aggiornati al',
    'polizia penitenziaria - previsti',
    'polizia penitenziaria - effettivi',
    'personale polizia penitenziaria aggiornato al',
    'amministrativi - effettivi',
    'amministrativi - previsti',
    'personale amministrativo aggiornato al',
    'numero complessivo',
    'numero non disponibili',
    'doccia',
    'bidet',
    'portatori di handicap',
    'servizi igienici con porta',
    'accensione luce autonoma',
    'prese elettriche',
    'data di aggiornamento spazi detentivi',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head(2)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude
0,MII152284,Genova Marassi,123.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,658,2025-06-24,...,0.0,128.0,175.0,3.0,213.0,100.0,0.0,2025-06-24,44.417913,8.951100
1,MII157783,Alba,112.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,55,2025-06-24,...,86.0,22.0,112.0,1.0,112.0,22.0,22.0,2025-06-24,44.691542,8.025636


In [50]:
df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


df_filtered.head()

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_64815/3401019820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude,scheda istituto
0,MII152284,Genova Marassi,123.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,658,2025-06-24,...,128.0,175.0,3.0,213.0,100.0,0.0,2025-06-24,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,112.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,55,2025-06-24,...,22.0,112.0,1.0,112.0,22.0,22.0,2025-06-24,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,212.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,219,483,1025,2025-06-24,...,272.0,184.0,0.0,327.0,325.0,325.0,2025-06-24,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,137.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,112,2025-06-24,...,20.0,20.0,0.0,50.0,40.0,40.0,2025-06-24,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,142.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,24,259,368,2025-06-24,...,192.0,42.0,0.0,247.0,228.0,0.0,2025-06-24,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [51]:
# Create a copy first to avoid the warning
df_filtered = df_filtered.copy()

# Calculate metrics using loc for proper assignment
df_filtered.loc[:, 'stanze_disponibili'] = df_filtered['numero complessivo'] - df_filtered['numero non disponibili']

# Create list of columns to process
metrics = {
    'detenuti_stanza': lambda x: (x['totale detenuti'] / x['stanze_disponibili']).round(2),
    'polizia_pers': lambda x:  (x['polizia penitenziaria - effettivi']/ x['totale detenuti']).round(2),
    'doccia_pers': lambda x:  (x['totale detenuti'] / x['doccia']).round(2),
    'bidet_pers': lambda x:  (x['totale detenuti'] / x['bidet']).round(2),
    'servizi_pers': lambda x:  (x['totale detenuti'] / x['servizi igienici con porta']).round(2),
    'luci_pers': lambda x:  (x['totale detenuti'] / x['accensione luce autonoma']).round(2),
    'prese_pers': lambda x: (x['totale detenuti'] / x['prese elettriche']).round(2)
}

# Apply calculations
for col, func in metrics.items():
    df_filtered.loc[:, col] = func(df_filtered).replace([np.inf, -np.inf], np.nan)

In [52]:
df_filtered.sample(4)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,longitude,scheda istituto,stanze_disponibili,detenuti_stanza,polizia_pers,doccia_pers,bidet_pers,servizi_pers,luci_pers,prese_pers
156,MII181371,Sulmona,90.0,Via Lamaccio n. snc - 67039 Sulmona,Casa di reclusione,523,0,523,471,2025-06-24,...,13.927974,"<a href=""https://www.giustizia.it/giustizia/pa...",368.0,1.28,0.54,5.61,1.33,1.28,1.28,6.04
8,MII158941,Ancona Montacuto,134.0,Via Montecavallo di Montacuto n.73/a - 60129 A...,Casa circondariale,256,0,256,343,2025-06-24,...,13.527880,"<a href=""https://www.giustizia.it/giustizia/pa...",111.0,3.09,0.40,3.36,3.50,3.09,3.40,3.40
106,MII179237,Opera,152.0,Via Camporgnago n.40 - 20090 Milano,Casa di reclusione - Iª casa,918,18,900,1367,2025-06-24,...,9.211149,"<a href=""https://www.giustizia.it/giustizia/pa...",800.0,1.71,0.37,5.94,2.19,1.67,6.05,5.79
137,MII180370,Roma Rebibbia Nuovo Complesso,147.0,Via Raffaele Majetti n.70 - 00156 Roma,Casa circondariale - Raffaele Cinotti,1171,103,1068,1567,2025-06-24,...,12.574066,"<a href=""https://www.giustizia.it/giustizia/pa...",570.0,2.75,0.42,13.17,1567.00,4.18,2.52,2.73


In [53]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [54]:
grouped_df = df.groupby('dati aggiornati al').sum(numeric_only=True).reset_index()
grouped_df

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2025-06-18,51284,4593,62691,30786,34035,3284,4086,942,1003,31914.0,2934.0,14354.0,13702.0,433.0,30390.0,24487.0,21218.0
203,2025-06-19,51284,4584,62656,30786,34035,3284,4086,942,1003,31914.0,2930.0,14354.0,13702.0,433.0,30390.0,24487.0,21218.0
204,2025-06-22,51276,4585,62728,30786,34035,3284,4086,942,1003,31913.0,2931.0,14353.0,13701.0,433.0,30389.0,24486.0,21217.0
205,2025-06-23,51276,4578,62708,30786,34035,3284,4086,942,1003,31913.0,2927.0,14353.0,13701.0,433.0,30389.0,24486.0,21217.0


In [55]:
grouped_df['posti disponibili'] = (grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']).round(0)
grouped_df['tasso di affollamento'] = (grouped_df['totale detenuti'] / grouped_df['posti disponibili'] * 100).round(4).astype(float)

grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,posti disponibili,tasso di affollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.1883
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.2567
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46755,132.2768
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46756,132.3103
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46758,132.2619


In [56]:
grouped_df = grouped_df[['dati aggiornati al', 'posti regolamentari', 'posti non disponibili', 'posti disponibili', 'totale detenuti', 'tasso di affollamento']]
# grouped_df['posti disponibili'] = grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']
# grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
0,2024-10-05,51196,4384,46812,61880,132.1883
1,2024-10-06,51196,4384,46812,61912,132.2567
2,2024-10-07,51196,4441,46755,61846,132.2768
3,2024-10-09,51195,4439,46756,61863,132.3103
4,2024-10-10,51195,4437,46758,61843,132.2619


In [57]:
grouped_df.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
205,2025-06-23,51276,4578,46698,62708,134.2841
206,2025-06-24,51276,4576,46700,62705,134.2719


In [58]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [59]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')
df.columns

Index(['id istituto', 'nome istituto', 'tasso di affollamento', 'indirizzo',
       'tipo istituto', 'posti regolamentari', 'posti non disponibili',
       'posti disponibili', 'totale detenuti', 'dati aggiornati al',
       'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi',
       'personale polizia penitenziaria aggiornato al',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'personale amministrativo aggiornato al', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'data di aggiornamento spazi detentivi',
       'latitudine', 'longitude', 'scheda istituto', 'stanze_disponibili',
       'detenuti_stanza', 'polizia_pers', 'doccia_pers', 'bidet_pers',
       'servizi_pers', 'luci_pers', 'prese_pers'],
      dtype='object')

In [60]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome istituto', 'totale detenuti', 'tasso di affollamento', 'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi', 'personale polizia penitenziaria aggiornato al', 'dati aggiornati al']]

df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']

df_polizia.head(2)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_64815/2362681573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante
0,Genova Marassi,658,123.0,336,319,2025-05-31,2025-06-24,17
1,Alba,55,112.0,107,80,2025-05-31,2025-06-24,27


In [61]:
df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)
df_polizia.head(2)


/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_64815/989766878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Genova Marassi,658,123.0,336,319,2025-05-31,2025-06-24,17,5.06
1,Alba,55,112.0,107,80,2025-05-31,2025-06-24,27,25.23


In [62]:
df_polizia_clean = df_polizia[df_polizia['totale detenuti'] != 0]
df_polizia_clean.sort_values('polizia penitenziaria - mancante percentuale', ascending=False).head(2)

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
138,Roma Rebibbia,279,94.0,189,131,2025-05-31,2025-06-24,58,30.69
120,Pescara,390,160.0,166,117,2025-05-31,2025-06-24,49,29.52


In [63]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso di affollamento'] > 120) & (df_polizia_clean['polizia penitenziaria - mancante percentuale'] > 20)].reset_index(drop=True)
critical_prisons

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Ancona Montacuto,343,134.0,175,136,2025-05-31,2025-06-24,39,22.29
1,Augusta,593,174.0,224,173,2025-05-31,2025-06-24,51,22.77
2,Rieti,501,173.0,175,128,2025-05-31,2025-06-24,47,26.86
3,Bolzano,109,124.0,77,61,2025-05-31,2025-06-24,16,20.78
4,Brescia Verziano,122,174.0,95,74,2025-05-31,2025-06-24,21,22.11
5,Cagliari Uta,691,123.0,394,313,2025-05-31,2025-06-24,81,20.56
6,Cassino,158,170.0,142,102,2025-05-31,2025-06-24,40,28.17
7,Matera,189,151.0,125,94,2025-05-31,2025-06-24,31,24.80
8,Pescara,390,160.0,166,117,2025-05-31,2025-06-24,49,29.52
9,Roma Regina Coeli,1105,193.0,480,351,2025-05-31,2025-06-24,129,26.88


In [64]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [65]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [66]:
df2.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
205,2025-06-23,51276,4578,46698,62708,134.2841
206,2025-06-24,51276,4576,46700,62705,134.2719


In [67]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'dati aggiornati al': 'Date', 'tasso di affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [68]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [69]:
df2.tail(2)

,Date,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso_affollamento_reale
205,2025-06-23,51276,4578,46698,62708,134.2841
206,2025-06-24,51276,4576,46700,62705,134.2719


In [70]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
275,2025-06-23,NaN,134.2841
276,2025-06-24,NaN,134.2719


In [71]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
275,2025-06-23,NaN,134.2841
276,2025-06-24,NaN,134.2719


In [72]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_64815/3090938612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_64815/3090938612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpo

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
275,2025-06-23,NaN,134.2841,120.694,134.2841
276,2025-06-24,NaN,134.2719,120.694,134.2719


In [73]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)

In [74]:
df_reale = filtered_df[['Date', 'tasso_affollamento_reale']]
df_reale = df_reale[df_reale['tasso_affollamento_reale'].notna()]
df_reale.head(2)

,Date,tasso_affollamento_reale
73,2024-10-05,132.1883
74,2024-10-06,132.2567


In [75]:
df_reale.to_csv('../outputs/viz/tasso_reale.csv', index=False)